## Computing phases in a ternary system
This method is based on the [paper](https://pubs.rsc.org/en/content/articlelanding/2019/sm/c8sm02045k#!divAbstract) which uses a convex envelope method but identifies the potential phases using a Laplacian based approach

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config InlineBackend.figure_format = 'svg'
import mpltern
from matplotlib import rc
rc('text', usetex=True)
import seaborn as sns
import matplotlib.pyplot as plt

import pdb
import numpy as np

import sys
if '../' not in sys.path:
    sys.path.append('../')

from solvers.utils import get_data

import warnings
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
/projects/academic/olgawodo/kiranvad/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/projects/academic/olgawodo/kiranvad/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# make a general grid
def makegridnd(num_points,dim):
    x = np.meshgrid(*[np.linspace(0.001, 1,num_points) for d in range(dim)])
    mesh = np.asarray(x)
    total = np.sum(mesh,axis=0)
    plane_mesh = mesh[:,np.isclose(total,1.0,atol=1e-2)]
    
    return plane_mesh

num_points = 40
dim = 4
grid = makegridnd(num_points,dim)

print(grid.shape)
                         

(4, 11480)


In [4]:
# get your pure points
di = np.diag_indices(dim)
pure_points = 0.001*np.ones((dim,dim))
pure_points[di]=1
print(pure_points)
verts = pure_points[:,:3].tolist()
print(verts)

[[1.    0.001 0.001 0.001]
 [0.001 1.    0.001 0.001]
 [0.001 0.001 1.    0.001]
 [0.001 0.001 0.001 1.   ]]
[[1.0, 0.001, 0.001], [0.001, 1.0, 0.001], [0.001, 0.001, 1.0], [0.001, 0.001, 0.001]]


In [5]:
def utri2mat(utri,dim):
    inds = np.triu_indices(dim,1)
    ret = np.zeros((dim, dim))
    ret[inds] = utri
    ret.T[inds] = utri
    return ret

In [6]:
M = np.ones(dim) 
chi = 3.10*np.ones(int(0.5*dim*(dim-1)))
CHI = utri2mat(chi, dim)
print(CHI)

[[0.  3.1 3.1 3.1]
 [3.1 0.  3.1 3.1]
 [3.1 3.1 0.  3.1]
 [3.1 3.1 3.1 0. ]]


In [7]:
from solvers import helpers

gmix = lambda x: helpers.flory_huggins(x, M, CHI,beta=1e-4)
energy = []
for i in range(grid.shape[1]):
    energy.append(gmix(grid[:,i]))


In [8]:
# Plot a convex triangulation in the ternary space
from solvers.helpers import get_ternary_coords
from scipy.spatial import ConvexHull

points = np.concatenate((grid[:-1,:].T,np.asarray(energy).reshape(-1,1)),axis=1)
hull = ConvexHull(points)
coords = np.asarray(grid)

In [9]:
simplices = []
for simplex in hull.simplices:
    point_class = np.sum(np.isclose(grid[:,simplex],0.001),axis=1)
    point_class = np.unique(dim - point_class)
    if (point_class==1).any():
        pass
    else:
        simplices.append(simplex)

In [10]:
from scipy.spatial.distance import pdist, euclidean, squareform
from scipy.sparse import csr_matrix
import pandas as pd
from scipy.sparse.csgraph import connected_components

def label_triangle(triangle):
    thresh = 5*euclidean(coords[:,0],coords[:,1])
    tri_coords = [coords[:,x] for x in triangle]
    dist = squareform(pdist(tri_coords,'euclidean'))
    adjacency = dist<thresh
    adjacency =  adjacency.astype(int)  
    graph = csr_matrix(adjacency)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    
    return n_components

def get_phase_diagram(simplices):
    num_comps = [label_triangle(triangle) for triangle in simplices]
    
    return num_comps
    
num_comps = get_phase_diagram(simplices)


In [11]:
# based on: https://stackoverflow.com/questions/39408794/python-3d-pyramid
%matplotlib widget
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import numpy as np

def get_faces(v):
    # generate list of sides' polygons of our pyramid
    verts = [ [v[0],v[1],v[3]], [v[1],v[2],v[3]],
     [v[0],v[2],v[3]], [v[0],v[1],v[2]]]
    return verts

In [12]:
def from4d23d(fourd_coords):
    x,y,z,w = fourd_coords
    u = y+0.5*(z+w)
    v = np.sqrt(3)*(z/2 + w/6) 
    w = np.sqrt(6)*(w/3)
    
    return [u,v,w]      

In [31]:
# determine points inside a polyhedron
from scipy.spatial import Delaunay

def inpolyhedron(ph,points):
    """
    Given a polyhedron vertices in `ph`, and `points` return 
    critera that each point is either with in or outside the polyhedron
    
    Both polyhedron and points should have the same shape i.e. num_points X num_dimensions
    
    Returns a boolian array : True if inside, False if outside
    
    """
    tri = Delaunay(ph)
    inside = Delaunay.find_simplex(tri,points)
    criteria = inside<0
    return ~criteria

def test_inpolyhedron(ph=None,points=None):
    if ph is None:
        ph = np.array([[0, 0, 0], [1, 0, 0], [1/2,np.sqrt(3)/2,0],  [1/2,np.sqrt(3)/6,np.sqrt(6)/3]])
        points = np.random.rand(30,3)
    criteria = inpolyhedron(ph, points)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(ph[:, 0], ph[:, 1], ph[:, 2],color='black')
    verts = get_faces(ph)
    ax.add_collection3d(Poly3DCollection(verts, facecolors='black', linewidths=0.5, edgecolors='black', alpha=.05))
    surf = ax.scatter3D(points[:, 0], points[:, 1], points[:, 2],c=criteria.astype(int), cmap='bwr')
    cbar = fig.colorbar(surf, shrink=0.5, aspect=5, ticks=[0,1])
    cbar.ax.set_yticklabels(['Outside', 'Inside'])
    plt.show()
    
plt.close()       
test_inpolyhedron()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
# lift simplex labels to delaunay
coords = np.asarray([from4d23d(point) for point in grid.T])
grid_delaunay = Delaunay(coords)
lifted_label = np.zeros(grid_delaunay.simplices.shape[0])
for label,simplex in zip(num_comps,simplices):
    lifted_simplex_vertices = np.asarray([coords[x,:] for x in simplex])
    inside = Delaunay.find_simplex(grid_delaunay,lifted_simplex_vertices)
    pdb.set_trace()
    lifted_label[inside[~inside<0]]=label

> <ipython-input-81-d6c704254829>(9)<module>()
-> lifted_label[inside[~inside<0]]=label
(Pdb) lifted_simplex_vertices
array([[0.02761538, 0.01594375, 0.04264624],
       [0.04042308, 0.03812732, 0.04264624],
       [0.07884615, 0.04552185, 0.06356112],
       [0.06603846, 0.03812732, 0.04264624]])
(Pdb) inside
array([   -1,    -1, 29113, 41442], dtype=int32)
(Pdb) grid_delaunay.simplices[inside[~inside<0]]
array([[1580, 1586, 1581,  801],
       [1591, 1587, 1586,  801]], dtype=int32)
(Pdb) quit()


BdbQuit: 

In [80]:
fig,ax = plt.subplots()
ax.hist(lifted_label)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
phase = np.zeros(coords.shape[0])
for i,simplex in zip(lifted_label,grid_delaunay.simplices):
    v = np.asarray([coords[x,:] for x in simplex])
    inside = inpolyhedron(v, coords)
    pdb.set_trace()
    phase[inside]=i

> <ipython-input-74-84b63b412fe9>(6)<module>()
-> phase[inside]=i
(Pdb) inside
array([False, False, False, ..., False, False, False])
(Pdb) coords[inside,:]
array([[0.39903846, 0.28954116, 0.56551809],
       [0.39903846, 0.30433021, 0.54460322],
       [0.38623077, 0.28214664, 0.54460322],
       [0.41184615, 0.28214664, 0.54460322]])
(Pdb) i
0.0
(Pdb) quit()


BdbQuit: 

In [61]:
def plot_polyhedron(ph):
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(ph[:, 0], ph[:, 1], ph[:, 2],color='black')
    verts = get_faces(ph)
    ax.add_collection3d(Poly3DCollection(verts, facecolors='black', linewidths=0.5, edgecolors='black', alpha=.05))
    plt.show()
    
plot_polyhedron(v)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
from matplotlib import colors
plt.close()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
v = np.array([[0, 0, 0], [1, 0, 0], [1/2,np.sqrt(3)/2,0],  [1/2,np.sqrt(3)/6,np.sqrt(6)/3]])
ax.scatter3D(v[:, 0], v[:, 1], v[:, 2],color='black')
verts = get_faces(v)
ax.add_collection3d(Poly3DCollection(verts, facecolors='black', linewidths=0.5, edgecolors='black', alpha=.05))

criteria = np.logical_and(grid[3,:]<0.51,phase>0)
cmap = colors.ListedColormap(['tab:red','tab:olive','tab:cyan','tab:purple'])
boundaries = np.linspace(1,5,5)
norm = colors.BoundaryNorm(boundaries, cmap.N)
surf = ax.scatter3D(coords[criteria, 0], coords[criteria, 1], coords[criteria, 2],\
                    c=phase[criteria],cmap=cmap,norm=norm)
cbar = fig.colorbar(surf, shrink=0.5, aspect=5, ticks=[1.5,2.5,3.5,4.5])
cbar.ax.set_yticklabels(['1-Phase', '2-Phase', '3-Phase','4-Phase'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
from solvers.phase import PhaseModelling

configuration = {'M':M, 'chi':chi}
pm = PhaseModelling(4,configuration, refine_simplices = False)
num_comps = pm.run()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [54]:
# try gluing simplies
from solvers.visuals import plot_4d_phase_simplex_addition
plt.close()
plot_4d_phase_simplex_addition(pm,sliceat=0.5)

> /projects/academic/olgawodo/kiranvad/Projects/Spring2020/PhaseModelling/solvers/visuals.py(37)plot_4d_phase_simplex_addition()
-> fig, axs = plt.subplots(2,2,subplot_kw={'projection': '3d'}, figsize=(8,8))
(Pdb) phase
array([0., 0., 0., ..., 0., 0., 3.])
(Pdb) phase[phase==1]
array([1., 1., 1., ..., 1., 1., 1.])
(Pdb) np.sum(phase==2)
1076
(Pdb) np.sum(phase==)
*** SyntaxError: invalid syntax
(Pdb) np.sum(phase==4)
2
(Pdb) np.sum(phase==1)
1334
(Pdb) quit()


BdbQuit: 